In [1]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [2]:
data_backup = data.iloc[3524:]
data = data.iloc[:3524]

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3524 entries, 0 to 3523
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    3524 non-null   object 
 1   open    3524 non-null   float64
 2   high    3524 non-null   float64
 3   low     3524 non-null   float64
 4   close   3524 non-null   float64
 5   volume  3524 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 165.3+ KB


### 'trend's for high & low

In [4]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition_high = seasonal_decompose(data['high'], model='additive', period=30)
trend_high = decomposition_high.trend

decomposition_low = seasonal_decompose(data['low'], model='additive', period=30)
trend_low = decomposition_low.trend

In [6]:
trend_high.tail(20)

3504    6389.730843
3505    6394.359847
3506    6398.507015
3507    6403.096509
3508    6408.385506
3509            NaN
3510            NaN
3511            NaN
3512            NaN
3513            NaN
3514            NaN
3515            NaN
3516            NaN
3517            NaN
3518            NaN
3519            NaN
3520            NaN
3521            NaN
3522            NaN
3523            NaN
Name: trend, dtype: float64

In [7]:
data['trend_high'] = trend_high
data['trend_low'] = trend_low

In [12]:
import pandas as pd

for i in range(1,21):
    data[f'trend_h+{i}'] = data['trend_high'].shift(i)
for i in range(1,21):
    data[f'trend_l+{i}'] = data['trend_low'].shift(i)
data.dropna(axis=0, inplace=True)

In [14]:
data.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'trend_high',
       'trend_low', 'trend_h+1', 'trend_h+2', 'trend_h+3', 'trend_h+4',
       'trend_h+5', 'trend_h+6', 'trend_h+7', 'trend_h+8', 'trend_h+9',
       'trend_h+10', 'trend_h+11', 'trend_h+12', 'trend_h+13', 'trend_h+14',
       'trend_h+15', 'trend_h+16', 'trend_h+17', 'trend_h+18', 'trend_h+19',
       'trend_h+20', 'trend_l+1', 'trend_l+2', 'trend_l+3', 'trend_l+4',
       'trend_l+5', 'trend_l+6', 'trend_l+7', 'trend_l+8', 'trend_l+9',
       'trend_l+10', 'trend_l+11', 'trend_l+12', 'trend_l+13', 'trend_l+14',
       'trend_l+15', 'trend_l+16', 'trend_l+17', 'trend_l+18', 'trend_l+19',
       'trend_l+20'],
      dtype='object')

In [15]:
for i, name in enumerate(data.columns):
    print(i, name)

0 date
1 open
2 high
3 low
4 close
5 volume
6 trend_high
7 trend_low
8 trend_h+1
9 trend_h+2
10 trend_h+3
11 trend_h+4
12 trend_h+5
13 trend_h+6
14 trend_h+7
15 trend_h+8
16 trend_h+9
17 trend_h+10
18 trend_h+11
19 trend_h+12
20 trend_h+13
21 trend_h+14
22 trend_h+15
23 trend_h+16
24 trend_h+17
25 trend_h+18
26 trend_h+19
27 trend_h+20
28 trend_l+1
29 trend_l+2
30 trend_l+3
31 trend_l+4
32 trend_l+5
33 trend_l+6
34 trend_l+7
35 trend_l+8
36 trend_l+9
37 trend_l+10
38 trend_l+11
39 trend_l+12
40 trend_l+13
41 trend_l+14
42 trend_l+15
43 trend_l+16
44 trend_l+17
45 trend_l+18
46 trend_l+19
47 trend_l+20


In [16]:
inputs_high = data.iloc[:, [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]]
target_high = data.iloc[:, [6]]

inputs_low = data.iloc[:, [28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47]]
target_low = data.iloc[:, [7]]

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor

X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(inputs_high, target_high, test_size=0.2, random_state=1, shuffle=False)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(inputs_low, target_low, test_size=0.2, random_state=1, shuffle=False)

from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_h = MinMaxScaler()
scaler_h.fit(X_train_h)
scaler_target_h = MinMaxScaler()
scaler_target_h.fit(y_train_h)

scaler_l = MinMaxScaler()
scaler_l.fit(X_train_l)
scaler_target_l = MinMaxScaler()
scaler_target_l.fit(y_train_l)

X_train_h_scaled = scaler_h.transform(X_train_h)
X_test_h_scaled = scaler_h.transform(X_test_h)
y_train_h_scaled = scaler_target_h.transform(y_train_h)
y_test_h_scaled = scaler_target_h.transform(y_test_h)

X_train_l_scaled = scaler_l.transform(X_train_l)
X_test_l_scaled = scaler_l.transform(X_test_l)
y_train_l_scaled = scaler_target_l.transform(y_train_l)
y_test_l_scaled = scaler_target_l.transform(y_test_l)

random_tree_high = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_high.fit(X_train_h_scaled, y_train_h_scaled)

random_tree_low = RandomForestRegressor(n_estimators=100, random_state=1)
random_tree_low.fit(X_train_l_scaled, y_train_l_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=1)

### data reading:

In [30]:
data_backup = pd.read_csv('data/^NDX_raw_data.csv')
data_backup.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [32]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition_high_backup = seasonal_decompose(data_backup['high'], model='additive', period=30)
trend_high_backup = decomposition_high_backup.trend

decomposition_low_backup = seasonal_decompose(data_backup['low'], model='additive', period=30)
trend_low_backup = decomposition_low_backup.trend

In [33]:
data_backup['trend_high'] = trend_high_backup
data_backup['trend_low'] = trend_low_backup

In [34]:
import pandas as pd

for i in range(1,21):
    data_backup[f'trend_h+{i}'] = data_backup['trend_high'].shift(i)
for i in range(1,21):
    data_backup[f'trend_l+{i}'] = data_backup['trend_low'].shift(i)
data_backup.dropna(axis=0, inplace=True)

In [35]:
inputs_high_backup = data_backup.iloc[:, [8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]]
inputs_low_backup = data_backup.iloc[:, [28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47]]

In [36]:
df_pred = dict()

scaler_input = MinMaxScaler()
data_backup_h_scaled = scaler_input.fit_transform(inputs_high_backup)
scaler_target = MinMaxScaler()
target_backup_c_scaled = scaler_target.fit_transform(data_backup[['trend_high']])

y_pred_h_backup = random_tree_high.predict(data_backup_h_scaled)
y_pred_h_backup = y_pred_h_backup.reshape(-1,1)
y_pred_h_backup = scaler_target.inverse_transform(y_pred_h_backup)

scaler_input = MinMaxScaler()
data_backup_l_scaled = scaler_input.fit_transform(inputs_low_backup)
scaler_target = MinMaxScaler()
target_backup_c_scaled = scaler_target.fit_transform(data_backup[['trend_low']])

y_pred_l_backup = random_tree_low.predict(data_backup_l_scaled)
y_pred_l_backup = y_pred_l_backup.reshape(-1,1)
y_pred_l_backup = scaler_target.inverse_transform(y_pred_l_backup)

df_pred = {'pred_h':y_pred_h_backup, 'pred_l':y_pred_l_backup}

In [40]:
data_backup.reset_index(inplace=True)

from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error

score_h = mean_absolute_error(data_backup[['high']], df_pred['pred_h'])
r2_h = r2_score(data_backup[['high']], df_pred['pred_h'])
data_backup['mae_h'] = score_h

score_l = mean_absolute_error(data_backup[['low']], df_pred['pred_l'])
r2_l = r2_score(data_backup[['low']], df_pred['pred_l'])
data_backup['mae_l'] = score_l

print(f'METRICS FOR NASDAQ INDEX:')
print(f'score high: {score_h}')
print(f'score low: {score_l}')

print(f'- R2 METRICS FOR NASDAQ INDEX:')
print(f'r2 high: {r2_h}')
print(f'r2 low: {r2_l}')

METRICS FOR NASDAQ INDEX:
score high: 93.9314122404026
score low: 100.61905608595995
- R2 METRICS FOR NASDAQ INDEX:
r2 high: 0.9982878824732738
r2 low: 0.997950199948707


/tmp/ipykernel_147289/3139995559.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_backup['mae_h'] = score_h
/tmp/ipykernel_147289/3139995559.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_backup['mae_l'] = score_l


In [38]:
remaining_columns = ('date','open','high','low','close','volume','trend_high','trend_low','mae_h','mae_l')

data_backup = data_backup[list(remaining_columns)]

y_pred_h_series = pd.Series(df_pred['pred_h'].reshape(1,-1)[0])
y_pred_l_series = pd.Series(df_pred['pred_l'].reshape(1,-1)[0])
data_backup['pred_high'] = y_pred_h_series
data_backup['pred_low'] = y_pred_l_series

data_backup.to_excel(f'data_w_pred_m5/index_w_pred_m5.xlsx')

/tmp/ipykernel_147289/2022941313.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_backup['pred_high'] = y_pred_h_series
/tmp/ipykernel_147289/2022941313.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_backup['pred_low'] = y_pred_l_series


In [39]:
import matplotlib.pyplot as plt

plt.figure(figsize=(200, 48))

plt.plot(data_backup['pred_high'], color='black')
plt.plot(data_backup['pred_low'], color='red')
plt.show()